<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-set-up-functions" data-toc-modified-id="Imports-and-set-up-functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports and set-up functions</a></span></li><li><span><a href="#Define-the-PDE-for-Sub-Surface-Temperature-(SST)" data-toc-modified-id="Define-the-PDE-for-Sub-Surface-Temperature-(SST)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define the PDE for Sub-Surface Temperature (SST)</a></span></li><li><span><a href="#Simulate-SST-data" data-toc-modified-id="Simulate-SST-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Simulate SST data</a></span><ul class="toc-item"><li><span><a href="#Use-2D-Navier-Stokes-solver-to-simulation-motion-vector" data-toc-modified-id="Use-2D-Navier-Stokes-solver-to-simulation-motion-vector-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Use 2D Navier-Stokes solver to simulation motion vector</a></span></li><li><span><a href="#Generate-simulated-SST-data" data-toc-modified-id="Generate-simulated-SST-data-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Generate simulated SST data</a></span></li></ul></li><li><span><a href="#Predict-future-SST-states" data-toc-modified-id="Predict-future-SST-states-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Predict future SST states</a></span><ul class="toc-item"><li><span><a href="#CDNN" data-toc-modified-id="CDNN-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>CDNN</a></span></li></ul></li></ul></div>

# Imports and set-up functions

In [3]:
import os

In [4]:
def dataDir(x):
    cwd = os.getcwd()
    
    return cwd + x 

# Define the PDE for Sub-Surface Temperature (SST)

From (Bézenac, Pajot, and Gallinari, 2019), Equation 3 (p. 2) gives the linear, elliptic PDE for advection-diffusion: 

$\begin{equation}
\frac{\partial I}{\partial t} + (\vec\omega . \nabla)I = D\nabla^2 I
\tag{1}
\label{eq1}
\end{equation}$

where $\vec\omega$ is the motion vector $\frac{\Delta x}{\Delta t}$, $D$ is the diffusion coefficient, $\nabla$ denotes the gradient operator, and $\nabla^2$ denotes the Laplacian operator.


**Theorem 1** states that, for any initial condition $I_0 \in L^1(\mathbb{R}^2)$ with $I_0(\pm \infty) = 0$, there exists a unique global solution $I(x, t)$ to $\eqref{eq1}$:

$\begin{equation}
I(x, t) = \int_{\mathbb{R}^2} k(x - w, y) I_0(y)dy
\tag{2}
\label{eq2}
\end{equation}$

Where $k(u, v) = \frac{1}{4\pi Dt}e^{-\frac{1}{4Dt}||u-v||^2}$ is a radial basis function kernel, or alternatively, a 2-dimensional Gaussian probability density with mean $u$ and variance $2Dt$.

All this means that if the initial condition $I_0$, the diffusion coefficient $D$, and the motion vector $\vec\omega$ were known, future temperatures could be predicted from past ones. Unfortunately in practice, none of these quantities are known; they must be estimated from the data. Below, we simulate test data and use it to estimate the motion vector $\vec\omega$ using a Convolution-Deconvolution Neural Network (CDNN). 

# Simulate SST data 

The motion vector $\vec\omega$ can be estimated using a Navier-Stokes PDE solver. The Navier-Stokes equations  mathematically express momentum balance and conservation of mass for Newtonian fluids, for example ocean water. The solution of the equations is a flow velocity, equivalent to the motion vector $\vec\omega$. It is a vector field—to every point in a fluid, at any moment in a time interval, it gives a vector whose direction and magnitude are those of the velocity of the fluid at that point in space and at that moment in time.

## Use 2D Navier-Stokes solver to simulation motion vector

## Generate simulated SST data

# Predict future SST states

## CDNN